# ILD CNN prediction part
## data set is testset
    the model is rebuilt from the JSON file and recompiled with the weights 
    the weights are loaded from ILD_CNN_model_weights
    

In [1]:
import argparse
import numpy as np
import sklearn.metrics as metrics
import cPickle as pickle
import json
from keras.models import model_from_json
import sys
import cv2

from keras.models import Sequential
from keras.utils import np_utils 
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D,AveragePooling2D
from keras.layers.advanced_activations import LeakyReLU

Using Theano backend.


In [2]:
# hinting the path into the python directory where my helper and model files reside
sys.path.insert(0, '../python')

In [4]:
import ild_helpers as H
import cnn_model as CNN

In [5]:
args         = H.parse_args()                          
train_params = {
     'do' : float(args.do) if args.do else 0.5,        
     'a'  : float(args.a) if args.a else 0.3,          # Conv Layers LeakyReLU alpha param [if alpha set to 0 LeakyReLU is equivalent with ReLU]
     'k'  : int(args.k) if args.k else 4,              # Feature maps k multiplier
     's'  : float(args.s) if args.s else 1,            # Input Image rescale factor
     'pf' : float(args.pf) if args.pf else 1,          # Percentage of the pooling layer: [0,1]
     'pt' : args.pt if args.pt else 'Avg',             # Pooling type: Avg, Max
     'fp' : args.fp if args.fp else 'proportional',    # Feature maps policy: proportional, static
     'cl' : int(args.cl) if args.cl else 5,            # Number of Convolutional Layers
     'opt': args.opt if args.opt else 'Adam',          # Optimizer: SGD, Adagrad, Adam
     'obj': args.obj if args.obj else 'ce',            # Minimization Objective: mse, ce
     'patience' : args.pat if args.pat else 5,         # Patience parameter for early stoping
     'tolerance': args.tol if args.tol else 1.005,     # Tolerance parameter for early stoping [default: 1.005, checks if > 0.5%]
     'res_alias': args.csv if args.csv else 'res'      # csv results filename alias
}

In [6]:
# load both the model and the weights

model = H.load_model()

In [7]:
model.compile(optimizer='Adam', loss=CNN.get_Obj(train_params['obj']))

In [8]:
X_predict = pickle.load( open( "../pickle/X_test.pkl", "rb" ) )

In [9]:
# adding a singleton dimension and rescale to [0,1]
X_predict = np.asarray(np.expand_dims(X_predict,1))/float(255)

In [11]:
# predict and check classification and probabilities are the same

classes = model.predict_classes(X_predict, batch_size=10)
proba = model.predict_proba(X_predict, batch_size=10)

6250/6250 [==============================] - 67s    


In [12]:
classes

array([4, 1, 4, ..., 2, 1, 1])

In [ ]:
proba

In [13]:
pickle.dump(classes, open( "../pickle/predicted_classes_test_set.pkl", "wb" ))
pickle.dump(proba, open( "../pickle/predicted_probabilities_test_set.pkl", "wb" ))